# Understanding retries
Here description of what this does.

In [1]:
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)
jobs = {}

status_order = {
    "finished"  : 0,
    "merging"   : 1,
    "cancelled" : 2,
    "failed"    : 3,
    "closed"    : 4,
    "undefined" : 5
}

In [2]:
def updatechain(group, currentid, depth):
    ### Update job chain information    
    
    if currentid not in jobs:
        return None
    
    group[9].append(currentid)
    if jobs[currentid][6] == "failed":
        depth += 1
        group[8] += 1
    maxdepthid = (depth, currentid)

    group[2] += jobs[currentid][5]
    if status_order[jobs[currentid][6]] < status_order[group[1]]:
        group[1] = jobs[currentid][6]
    for child in jobs[currentid][4]:
        tmpdepthid = updatechain(group, child, depth)
        if tmpdepthid != None and tmpdepthid[0] > maxdepthid[0]:
            maxdepthid = tmpdepthid
    return maxdepthid

### select period

In [3]:
start_date = datetime.datetime(2017, 8, 1)
#curre_date = datetime.datetime(2017, 8, 10, 23, 59, 59)
curre_date = datetime.datetime(2017, 8, 28, 23, 59, 59)
#curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days
timemargin = datetime.timedelta(1)

### selecting indices to scan

In [4]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-08-01,jobs_archive_2017-08-02,jobs_archive_2017-08-03,jobs_archive_2017-08-04,jobs_archive_2017-08-05,jobs_archive_2017-08-06,jobs_archive_2017-08-07,jobs_archive_2017-08-08,jobs_archive_2017-08-09,jobs_archive_2017-08-10,jobs_archive_2017-08-11,jobs_archive_2017-08-12,jobs_archive_2017-08-13,jobs_archive_2017-08-14,jobs_archive_2017-08-15,jobs_archive_2017-08-16,jobs_archive_2017-08-17,jobs_archive_2017-08-18,jobs_archive_2017-08-19,jobs_archive_2017-08-20,jobs_archive_2017-08-21,jobs_archive_2017-08-22,jobs_archive_2017-08-23,jobs_archive_2017-08-24,jobs_archive_2017-08-25,jobs_archive_2017-08-26,jobs_archive_2017-08-27,jobs_archive_2017-08-28


### running the scan

In [5]:
job_query = {
    "size": 0,
    "_source": [ "corecount", "exeerrorcode", "pandaid", "failedattempt",
                "child_ids", "wall_time", "jobstatus", "processingtype", "actualcorecount",
                "creationtime", "endtime"],
    'query':{
#        "match_all": {}
             'bool':{
#                    'must':[
#                         { "term": {"jobstatus": "finished" } },
#                         { "term": {"prodsourcelabel": "managed" } }
#                     ],
#                    'should':[
                       # { "term": {"prodsourcelabel": "rc_test" } },
                       # { "term": {"prodsourcelabel": "prod_test" } },
                       # { "term": {"prodsourcelabel": "managed" } }
#                        { "term": {"jobstatus": "finished" } },
#                        { "term": {"jobstatus": "failed" } },
#                    ]
             }
    }
                
}

scroll = scan(client=es, index=job_indices, query=job_query, scroll='1000000m', timeout="1000000m", size=10000)

count = 0
for res in scroll:
    count += 1
    
    #print(res)
    #if count>10000: break
    
    r = res['_source']
    
    cores = r['corecount']
    errorcode = r['exeerrorcode']
    pandaid = r['pandaid']
    failedattempt = r['failedattempt']
    if 'child_ids' in r:
        child_ids = r['child_ids']
    else:
        child_ids=[]
    wall_time = r['wall_time']
    jobstatus = r['jobstatus']
    processingtype = r['processingtype']
    actualcorecount = r['actualcorecount']
    creationtime = r['creationtime']
    endtime = r['endtime']
    
    if actualcorecount != None:
        wall_time *= actualcorecount
    elif cores != None:
        wall_time *= cores
    
    if not count%100000: 
        print(count)
        #print(data)
    
    jobs[pandaid]=[True, cores, errorcode, failedattempt, child_ids, wall_time, jobstatus, processingtype, creationtime, endtime]
    #data.append(doc)

#job=pd.DataFrame(data,columns=['initialjob', 'cores', 'errorcode', 'pandaid', 'failedattempt', 'child_ids', 'wall_time', 'jobstatus', 'processingtype'])
#del data
#job.head(100)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

### Debug

In [6]:
'''
jobs.clear()
jobs[0]=[True, 1, 0, 0, [1], 1, "failed", "debug"]
jobs[1]=[True, 1, 0, 0, [2], 1, "failed", "debug"]
jobs[2]=[True, 1, 0, 0, [3,5], 1, "failed", "debug"]
jobs[3]=[True, 1, 0, 0, [4], 1, "failed", "debug"]
jobs[4]=[True, 1, 0, 0, [], 1, "finished", "debug"]
jobs[5]=[True, 1, 0, 0, [6], 1, "failed", "debug"]
jobs[6]=[True, 1, 0, 0, [7], 1, "failed", "debug"]
jobs[7]=[True, 1, 0, 0, [], 1, "failed", "debug"]
print(jobs)
'''

'\njobs.clear()\njobs[0]=[True, 1, 0, 0, [1], 1, "failed", "debug"]\njobs[1]=[True, 1, 0, 0, [2], 1, "failed", "debug"]\njobs[2]=[True, 1, 0, 0, [3,5], 1, "failed", "debug"]\njobs[3]=[True, 1, 0, 0, [4], 1, "failed", "debug"]\njobs[4]=[True, 1, 0, 0, [], 1, "finished", "debug"]\njobs[5]=[True, 1, 0, 0, [6], 1, "failed", "debug"]\njobs[6]=[True, 1, 0, 0, [7], 1, "failed", "debug"]\njobs[7]=[True, 1, 0, 0, [], 1, "failed", "debug"]\nprint(jobs)\n'

### Find initial jobs

In [7]:
count_child = 0
count_parent = 0
for key,job in jobs.items():
    if len(job[4]) > 1 and job[7] == "simul":
        count_child += 1
        if (count_child < 0):
            print('Multi-child : '+str(key))

    for child in job[4]:
        if child in jobs:
            if jobs[child][0] == False:
                count_parent += 1
                if (count_parent < 100):
                    print('Multi-parent : '+str(key)+' '+str(child)+job[7])
            jobs[child][0] = False


Multi-parent : 3556770993 3557707874panda-client-0.5.86-jedi-run
Multi-parent : 3556773508 3566178828panda-client-0.5.86-jedi-run
Multi-parent : 3556774281 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774286 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774287 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774291 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774296 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774299 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774303 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774307 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774309 3558163771panda-client-0.5.86-jedi-run
Multi-parent : 3556774315 3558163772panda-client-0.5.86-jedi-run
Multi-parent : 3556774317 3558163772panda-client-0.5.86-jedi-run
Multi-parent : 3556774319 3558163772panda-client-0.5.86-jedi-run
Multi-parent : 3556774322 3558163772panda-client-0.5.86-jedi-run
Multi-parent : 3556774325

### Make a jobgroup from starting from an initial job

In [8]:
import re
jobgroups = [] # initial_id, status, walltime, depth, failedattempts, core, errorcode, processingtype, #totalfailedattempts, joblist, sharedjob
for key,job in jobs.items():
    #jobstartstr = re.sub('\d+(st|nd|rd|th)', lambda m: m.group()[:-2].zfill(2), job[8])
    #jobendstr   = re.sub('\d+(st|nd|rd|th)', lambda m: m.group()[:-2].zfill(2), job[9])
    jobstart = datetime.datetime.strptime(job[8], '%Y-%m-%dT%X')
    jobend   = datetime.datetime.strptime(job[9], '%Y-%m-%dT%X')
    if job[0] == True and \
       jobstart > start_date + timemargin and jobend > start_date + timemargin and \
       jobstart < curre_date - timemargin and jobend < curre_date - timemargin:
           jobgroups.append([key, "undefined", 0, 0, 0, job[1], job[2], job[7], 0, [], 0])
print('Number of job groups')
print(len(jobgroups))

Number of job groups
28465948


### Follow job chains

In [ ]:
count = 0
for group in jobgroups:
    maxdepthid = updatechain(group, group[0], 0)
    if maxdepthid != None:
        group[3] = maxdepthid[0]
        count += 1
    if not count%100000: 
        print(count)
    if maxdepthid[0] in jobs:
        group[4] = jobs[maxdepthid[0]][3]

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

### Count shared job

In [ ]:
for group in jobgroups:
    for jobid in group[9]:
        for group2 in jobgroups:
            if group is not group2:
                for jobid2 in group2[9]:
                    if jobid == jobid2:
                        group[10] += 1

nchainperactivity = {}
nchainperactivity_withshared = {}
for group in jobgroups:
    if group[7] not in nchainperactivity:
        nchainperactivity[group[7]] = 0
        nchainperactivity_withshared[group[7]] = 0
    nchainperactivity[group[7]] += 1
    if group[10] > 0:
        nchainperactivity_withshared[group[7]] += 1

print('Number of chains')
print(nchainperactivity)
print('Number of chains with shared jobs')
print(nchainperactivity_withshared)

### Print result

In [ ]:
count = 0
count2 = 0
data_finished = {}
data_failed = {}
data_closed = {}
data_cancelled = {}
data_merging = {}
data_unknown = {}
fill_number = True
for group in jobgroups:
    if group[10] > 0:
        continue

    count += 1
    if group[7] not in data_finished:
        data_finished[group[7]] = []
        data_failed[group[7]] = []
        data_closed[group[7]] = []
        data_cancelled[group[7]] = []
        data_merging[group[7]] = []
        data_unknown[group[7]] = []
    if fill_number:
        if group[1] == "finished":
            data_finished[group[7]].append([group[3], 1])
        elif group[1] == "failed":
            data_failed[group[7]].append([group[3], 1])
            if group[7] == 'deriv' and group[3] == 20:
                count2 += 1
                if count2 < 100:
                    print(str(group[0])+' '+str(group[3])+' '+str(group[2])+' '+str(group[8]))
        elif group[1] == "closed":
            data_closed[group[7]].append([group[3], 1])
        elif group[1] == "cancelled":
            data_cancelled[group[7]].append([group[3], 1])
        elif group[1] == "merging":
            data_merging[group[7]].append([group[3], 1])
        else:
            data_unknown[group[7]].append([group[3], 1])

    else:
        if group[1] == "finished":
            data_finished[group[7]].append([group[3], group[2]])
        elif group[1] == "failed":
            data_failed[group[7]].append([group[3], group[2]])
        elif group[1] == "closed":
            data_closed[group[7]].append([group[3], group[2]])
        elif group[1] == "cancelled":
            data_cancelled[group[7]].append([group[3], group[2]])
        elif group[1] == "merging":
            data_merging[group[7]].append([group[3], group[2]])
        else:
            data_unknown[group[7]].append([group[3], group[2]])

    #data[group[7]].append([group[4], group[2]])
    if count < 10:
      print(group)

jobdata = {}
jobdatasum = {}
pd.set_option('display.max_rows', 1000)

print('=== finished ===')
for key,datum in data_finished.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])

print('=== failed ===')
for key,datum in data_failed.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== closed ===')
for key,datum in data_closed.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== cancelled ===')
for key,datum in data_cancelled.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== merging ===')
for key,datum in data_merging.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== unknown ===')
for key,datum in data_unknown.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])